#  EPA-122A *Spatial* Data Science


## Lab 1 - part 1: Data, Grammar and Engineering (Pandas)

**TU Delft**<br>
**Q2 2024**<br>
**Instructors:** Giacomo Marangoni, Theodoros Chatzivasileiadis <br>

---


## Table of Contents

* [Learning Goals](#section0)
* [Data Munging](#section1)
    * [Dataset](#section_1_1)
    * [Data, Sliced and Diced](#section_1_2)
* [Visual Exploration](#section2)
    * [Histogram](#section_2_1)
    * [Kernel Density Plots](#section_2_2)
    * [Line and Bar Plots](#section_2_3)
* [(Un)tidy Data](#section3)
    * [Grouping, transforming, aggregating](#section_3_1)

## Learning Goals <a class="anchor" id="section0"></a>

- Obtain the basic tools to manipulate, investigate and visualise the data.
- Understand the concept of tidy/untidy data and how to prepare the data.

This notebook covers the basic tools of data analysis that are **needed** for the course.

## Data Munging <a class="anchor" id="section1"></a>

Real world datasets are messy. There is no way around it: datasets have "holes" (missing data), the amount of formats in which data can be stored is endless, and the best structure to share data is not always the optimum to analyze them, hence the need to [munge](https://dictionary.reference.com/browse/munge) them. As has been correctly pointed out in many outlets ([e.g.](https://www.nytimes.com/2014/08/18/technology/for-big-data-scientists-hurdle-to-insights-is-janitor-work.html?_r=0)), much of the time [spent](https://twitter.com/BigDataBorat/status/306596352991830016) in what is called (Geo-)Data Science is related not only to sophisticated modeling and insight, but has to do with much more basic and less exotic tasks such as obtaining data, processing, turning them into a shape that makes analysis possible, and exploring it to get to know their basic properties.

For how labor intensive and relevant this aspect is, there is surprisingly very little published on patterns, techniques, and best practices for quick and efficient data cleaning, manipulation, and transformation. In this session, you will use a few real world datasets and learn how to process them into Python so they can be transformed and manipulated, if necessary, and analyzed. For this, we will introduce some of the bread and butter of data analysis and scientific computing in Python. These are fundamental tools that are constantly used in almost any task relating to data analysis.

This notebook covers the basic and the content that is expected to be learnt by every student. We use a prepared dataset that saves us much of the more intricate processing that goes beyond the introductory level the session is aimed at. As a companion to this introduction, there is an additional notebook (see link on the website page for Lab 01) that covers how the dataset used here was prepared from raw data downloaded from the internet, and includes some additional exercises you can do if you want dig deeper into the content of this lab.

In this notebook, we discuss several patterns to clean and structure data properly, including tidying, subsetting, and aggregating; and we finish with some basic visualization. An additional extension presents more advanced tricks to manipulate tabular data.

Before we get our hands data-dirty, let us import all the additional libraries we will need, so we can get that out of the way and focus on the task at hand:

In [ ]:
# This ensures visualizations are plotted inside the notebook
%matplotlib inline
import matplotlib
import os  # This provides several system utilities
import pandas as pd  # This is the workhorse of data munging in Python
import numpy as np  # This is for general numerical operations
import seaborn as sns  # This allows us to efficiently and beautifully plot
import os  # This provides several system utilities


### Dataset <a class="anchor" id="section_1_1"></a>

We will be exploring some of the characteristics of the population in The Hague. To do that, we will use a dataset that contains population counts, split by ethnic origin: https://denhaag.incijfers.nl/jive. Let us first set the path to the file where we store the data we will use:

In [ ]:
# Important! You need to specify the path to the data in *your* machine
# If you have placed the data folder in the same directory as this notebook,
# you would do:

f = "data/DenHaag"  # Path to file containing the table

# Check to see if the path is correct and works. If you have set it
# correctly, you should obtain the following list
os.listdir(f)

**IMPORTANT**: the path above might look different in your computer.

To read a "comma separated values" (`.csv`) file, we can run:

In [ ]:
# We are going to work with DenHaag_.csv. Let's adjust the path:
f = "data/DenHaag/DenHaag_.csv"
db = pd.read_csv(f, index_col="Buurten")  # Read the table in

Let us stop for a minute to learn how we have read the file. Here are the main aspects to keep in mind:

* We are using the method `read_csv` from the `pandas` library, which we have imported with the alias `pd`.
* In this form, all that is required is to pass the path to the file we want to read, which in this case we have created by concatenating two strings. We can see the full path we have used:


In [ ]:
f

* the `sep` argument defines the seperator. It is default set to `,` which is also the standard deperator in a csv file. In our case the dataset is seperated by `;` however.
* The argument `index_col` is not strictly necessary but allows us to choose one of the columns as the index of the table. More on indices below.
* We are using `read_csv` because the file we want to read is in the `csv` format. However, `pandas` allows for many more formats to be read (and written, just replace `read` by `to`! For example, `read_csv` reads in, `to_csv` writes out). A full list of formats supported may be found [here](https://pandas.pydata.org/pandas-docs/version/0.18.1/io.html).

### Data, Sliced and Diced <a class="anchor" id="section_1_2"></a>

Now we are ready to start playing and interrogating the dataset! What we have at our fingertips is a table that summarizes, for each of the buurtens in The Hague, how many people live in each, by the region of the world where they were born. Now, let us learn a few cool tricks built into `pandas` that work out-of-the box with a table like ours.

* Inspecting what it looks like. We can check the top (bottom) X lines of the table by passing X to the method `head` (`tail`). For example, for the top/bottom five lines:

In [ ]:
db.head()

In [ ]:
db.tail()

- Now lets get an overview of data types

In [ ]:
db.info()

In [ ]:
db.describe()


Note how the output is also a `DataFrame` object, so you can do with it the same things you would with the original table (e.g. writing it to a file). In this case, the summary might be better presented if the table is "transposed":

In [ ]:
db.describe().T

* Equally, common descriptive statistics are also available:

In [ ]:
# Obtain minimum values for each table
db.min()

In [ ]:
# Obtain the minimum population share of Marokaans
db["% Marokkaans|2023"].min()

In [ ]:
# Obtain standard deviation for the row `117 De Rivieren`,
# Does this value have any informative value for us?
db.loc["117 De Rivieren", :].std()

* Creation of new variables: we can generate new variables by applying operations on existing ones. For example, we can calculate the total population by area. Here is a couple of ways to do it:

In [ ]:
# One shot
total = db.sum(axis=1)
# Print the top of the variable
total.head()

In [ ]:
# Summing up a subsection of variables:
Non_Dutch = (
    db["% Turks|2023"]
    + db["% Marokkaans|2023"]
    + db["% Surinaams|2023"]
    + db["% Antilliaans|2023"]
    + db["% Overig niet-westers|2023"]
    + db["% Westers|2023"]
)
# Print the top of the variable
Non_Dutch.head()

Note how we are using the command `sum`, just like we did with `max` or `min` before but, in this case, we are not applying it over columns (e.g. the max of each column), but over rows, so we get the total sum of populations by areas.

Once we have created the variable, we can make it part of the table:

In [ ]:
db["% Non-Dutch | 2023"] = Non_Dutch
db.head()

* Assigning new values: we can easily generate new variables with scalars, and modify those.

In [ ]:
# New variable with all ones
db["ones"] = 1
db.head()

And we can modify specific values too:

In [ ]:
db.loc["03 Scheveningen Badplaats", "ones"] = 3
db.head()

* Permanently deleting variables is also within reach of one command:

In [ ]:
del db["ones"]
db.head()

* Index-based querying.

We have already seen how to subset parts of a `DataFrame` if we know exactly which bits we want. For example, if we want to extract the total population and the share of Turkish people living in the first four neighbourhoods in the table, we use `loc` with lists:

In [ ]:
total_share_Turkish_first4 = db.loc[
    [
        "01 Oud Scheveningen",
        "02 Vissershaven",
        "03 Scheveningen Badplaats",
        "04 Visserijbuurt",
    ],
    ["Aantal inwoners per 1-1|2023", "% Turks|2023"],
]

total_share_Turkish_first4

* Querying based on conditions.

However, sometimes, we do not know exactly which observations we want, but we do know what conditions they need to satisfy (e.g. areas with more than 2,000 inhabitants). For these cases, `DataFrames` support selection based on conditions. Let us see a few examples. Suppose we want to select...

*... areas with more than 2,500 people in Total*:

In [ ]:
m2500 = db.loc[db["Aantal inwoners per 1-1|2023"] > 2500, :]
m2500

*... areas where there are no more than 10 people aged above 80.*:

In [ ]:
nm10_above_eighty = db.loc[db["80 jaar en ouder|2023"] <= 10, :]
nm10_above_eighty

*... areas with exactly nine person people aged above 80*:

In [ ]:
nine_above_eighty = db.loc[db["80 jaar en ouder|2023"] == 9, :]
nine_above_eighty

**Pro-tip**: these queries can grow in sophistication with almost no limits. For example, here is a case where we want to find out the areas where European population is less than half the population:

In [ ]:
nm_half_underaged = db.loc[
    (
        (db["0 t/m 4 jaar|2023"] + db["5 t/m 14 jaar|2023"] + db["15 t/m 19 jaar|2023"])
        / db["Aantal inwoners per 1-1|2023"]
    )
    < 0.5,
    :,
]
nm_half_underaged

* Combining queries.

Now all of these queries can be combined with each other, for further flexibility. For example, imagine we want areas with more than 25 toddlers but less than 1,500 in total:

In [ ]:
m25Toddlers_nm1500Total = db.loc[
    (db["0 t/m 4 jaar|2023"] > 25) & (db["Aantal inwoners per 1-1|2023"] < 1500), :
]
m25Toddlers_nm1500Total

* Sorting.

Among the many operations `DataFrame` objects support, one of the most useful ones is to sort a table based on a given column. For example, imagine we want to sort the table by Toddlers:

In [ ]:
db_Toddlers_sorted = db.sort_values("0 t/m 4 jaar|2023", ascending=False)
db_Toddlers_sorted.head()

If you inspect the help of `db.sort_values`, you will find that you can pass more than one column to sort the table by. This allows you to do so-called hiearchical sorting: sort first based on one column, if equal then based on another column, etc.

## Visual Exploration <a class="anchor" id="section2"></a>

The next step to continue exploring a dataset is to get a feel for what it looks like, visually. We have already learnt how to unconver and inspect specific parts of the data, to check for particular cases we might be intersted in. Now we will see how to plot the data to get a sense of the overall distribution of values. For that, we will be using the Python library [`seaborn`](http://stanford.edu/~mwaskom/software/seaborn/index.html).

### Histograms <a class="anchor" id="section_2_1"></a>

One of the most common graphical devices to display the distribution of values in a variable is a histogram. Values are assigned into groups of equal intervals, and the groups are plotted as bars rising as high as the number of values into the group.

A histogram is easily created with the following command. In this case, let us have a look at the shape of the overall population:

In [ ]:
_ = sns.displot(db["Aantal inwoners per 1-1|2023"], kde=False)

Note we are using `sns` instead of `pd`, as the function belongs to `seaborn` instead of `pandas`.

We can quickly see most of the areas contain somewhere between 1,200 and 1,700 people, approx. However, there are a few areas that have many more, even up to 3,500 people.

An additional feature to visualize the density of values is called `rug`, and adds a little tick for each value on the horizontal axis:

In [ ]:
_ = sns.displot(db["Aantal inwoners per 1-1|2023"], kde=False, rug=True)

### Kernel Density Plots <a class="anchor" id="section_2_2"></a>

Histograms are useful, but they are artificial in the sense that a continuous variable is made discrete by turning the values into discrete groups. An alternative is kernel density estimation (KDE), which produces an empirical density function:

In [ ]:
_ = sns.kdeplot(db["Aantal inwoners per 1-1|2023"], fill=True)

### Line and Bar plots <a class="anchor" id="section_2_3"></a>

Another very common way of visually displaying a variable is with a line or a bar chart. For example, if we want to generate a line plot of the (sorted) total population by area:

In [ ]:
_ = db["Aantal inwoners per 1-1|2023"].sort_values(ascending=False).plot()

For a bar plot all we need to do is to change an argument of the call:

In [ ]:
_ = db["Aantal inwoners per 1-1|2023"].sort_values(ascending=False).plot(kind="bar")

Note that the large number of areas makes the horizontal axis unreadable. We can try to turn the plot around by displaying the bars horizontally (see how it's just changing `bar` for `barh`). To make it readable, let us expand the plot's height:

In [ ]:
_ = db["Aantal inwoners per 1-1|2023"].sort_values().plot(kind="barh", figsize=(6, 20))

## (Un)tidy data <a class="anchor" id="section3"></a>

> *Happy families are all alike; every
unhappy family is unhappy in its own
way.*

> Leo Tolstoy.

Once you can read your data in, explore specific cases, and have a first visual approach to the entire set, the next step can be preparing it for more sophisticated analysis. Maybe you are thinking of modeling it through regression, or on creating subgroups in the dataset with particular characteristics, or maybe you simply need to present summary measures that relate to a slightly different arrangement of the data than you have been presented with.

For all these cases, you first need what statistician, and general R wizard, Hadley Wickham calls *"tidy data"*. The general idea to "tidy" your data is to convert them from whatever structure they were handed in to you into one that allows convenient and standardized manipulation, and that supports directly inputting the data into what he calls "*tidy*" analysis tools. But, at a more practical level, what is exactly *"tidy data"*? In Wickham's own words:

> *Tidy data is a standard way of mapping the meaning of a dataset to its structure. A dataset is
messy or tidy depending on how rows, columns and tables are matched up with observations,
variables and types.*

He then goes on to list the three fundamental characteristics of *"tidy data"*:

1. Each variable forms a column.
1. Each observation forms a row.
1. Each type of observational unit forms a table.

If you are further interested in the concept of *"tidy data"*, I recommend you check out the [original paper](https://www.jstatsoft.org/v59/i10/) (open access) and the [public repository](https://github.com/hadley/tidy-data) associated with it.

Let us bring in the concept of "*tidy data*" to our own The Hague dataset. First, remember its structure:

In [ ]:
db.head()

Thinking through *tidy* lenses, this is not a tidy dataset. It is not so for each of the three conditions:

* Starting by the last one (*each type of observational unit forms a table*), this dataset actually contains not one but two observational units: the different areas of The Hague, captured by `Buurten`; *and* subgroups of an area. To *tidy* up this aspect, we can create two different tables:

In [ ]:
# Assign column `Aantal inwoners per 1-1|2023` into its own as a single-column table
db_totals = db[["Aantal inwoners per 1-1|2023"]]
db_totals.head()

In [ ]:
# Create a table `db_subgroups` that contains every column in `db` without `Aantal inwoners per 1-1|2023`
db_subgroups = db.drop("Aantal inwoners per 1-1|2023", axis=1)
db_subgroups.head()

Note we use `drop` to exclude `Aantal inwoners per 1-1|2023`, but we could also use a list with the names of all the columns to keep. Additionally, notice how, in this case, the use of `drop` (which leaves `db` untouched) is preferred to that of `del` (which permanently removes the column from `db`).

At this point, the table `db_totals` is tidy: every row is an observation, every column is a variable, and there is only one observational unit in the table.

The other table (`db_subgroups`), however, is not entirely tidied up yet: there is only one observational unit in the table, true; but every row is not an observation, and there are variable values as the names of columns (in other words, every column is not a variable). To obtain a fully tidy version of the table, we need to re-arrange it in a way that every row is a population subgroup in an area, and there are three variables: `Buurten`, age subgroup, and count (or frequency).

Because this is actually a fairly common pattern, there is a direct way to solve it in `pandas`:

In [ ]:
tidy_subgroups = db_subgroups.stack()
tidy_subgroups.head()

The method `stack`, well, "stacks" the different columns into rows. This fixes our "tidiness" problems but the type of object that is returning is not a `DataFrame`:

In [ ]:
type(tidy_subgroups)

It is a `Series`, which really is like a `DataFrame`, but with only one column. The additional information (`Buurten` and age group) are stored in what is called an multi-index. We will skip these for now, so we would really just want to get a `DataFrame` as we know it out of the `Series`. This is also one line of code away:

In [ ]:
# Unfold the multi-index into different, new columns
tidy_subgroupsDF = tidy_subgroups.reset_index()
tidy_subgroupsDF.head()

To which we can apply to renaming to make it look better:

In [ ]:
tidy_subgroupsDF = tidy_subgroupsDF.rename(columns={"level_1": "Subgroup", 0: "Freq"})
tidy_subgroupsDF.head()

Now our table is fully tidied up!

### Grouping, transforming, aggregating <a class="anchor" id="section_3_1"></a>

One of the advantage of tidy datasets is they allow to perform advanced transformations in a more direct way. One of the most common ones is what is called "group-by" operations. Originated in the world of databases, these operations allow you to group observations in a table by one of its labels, index, or category, and apply operations on the data group by group.

For example, given our tidy table with population subgroups, we might want to compute the total sum of population by each group. This task can be split into two different ones:

* Group the table in each of the different subgroups.
* Compute the sum of `Freq` for each of them.

To do this in `pandas`, meet one of its workhorses, and also one of the reasons why the library has become so popular: the `groupby` operator.

In [ ]:
pop_grouped = tidy_subgroupsDF.groupby("Subgroup")
pop_grouped

The object `pop_grouped` still hasn't computed anything, it is only a convenient way of specifying the grouping. But this allows us then to perform a multitude of operations on it. For our example, the sum is calculated as follows:

In [ ]:
pop_grouped.sum()

Similarly, you can also obtain a summary of each group:

In [ ]:
pop_grouped.describe()

We will not get into it today as it goes beyond the basics we want to cover, but keep in mind that `groupby` allows you to not only call generic functions (like `sum` or `describe`), but also your own functions. This opens the door for virtually any kind of transformation and aggregation possible.
